In [16]:
pip install mariadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mariadb: filename=mariadb-1.1.10-cp312-cp312-macosx_11_0_arm64.whl size=79597 sha256=80eba53e3f6da03c4cbcf25fc7d970281936509102e34e14013d13bf7490ec0d
  Stored in directory: /Users/sungjae/Library/Caches/pip/wheels/49/11/3b/f7c4af9d972d01f5082482d36f58a6fb5fc86a64abb2d247e2
Successfully built mariadb
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import requests
# import json
# import re

# headers = {
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
# }

# # 음식점 정보를 저장할 리스트
# restaurants = []

# # 페이지별로 데이터 요청 및 처리
# for page in range(1, 9):
#     url = f'https://map.naver.com/p/api/search/allSearch?query=%EB%8F%99%EA%B5%AD%EB%8C%80+%EC%A3%BC%EB%B3%80+%EC%9D%8C%EC%8B%9D%EC%A0%90&type=all&searchCoord=126.97825%3B37.566551&page={page}'
#     # API 요청
#     res = requests.get(url, headers=headers)
#     print(res.status_code)
#     data = res.json()

#     # 음식점 정보 추출
#     if 'result' in data and 'place' in data['result']:
#         for item in data['result']['place']['list']:
#             name = item.get('name')
#             categories = item.get('category', [])

#             # '음식점'이 포함된 경우 두 번째 카테고리 선택
#             if categories and categories[0] == '음식점' and len(categories) > 1:
#                 category = categories[1]
#             else:
#                 category = categories[0] if categories else '카테고리 없음'

#             phone = item.get('tel', '전화번호 없음')
#             address = item.get('roadAddress', '주소 없음')
#             image = item.get('thumUrl', '이미지 없음')

#             # 메뉴 정보가 있는 경우
#             menus = []  # menus 변수를 정의합니다.
#             menu_info = item.get('menuInfo', '')  # 메뉴 정보가 있는 경우만 가져옵니다.

#             if menu_info:
#                 menu_list = menu_info.split('|')
#                 for menu_item in menu_list:
#                     # 가격 정보 추출을 위한 정규 표현식
#                     match = re.search(r'(\d{1,3}(?:,\d{3})*)$', menu_item.strip())
#                     if match:
#                         price = match.group(0)
#                         description = menu_item[:match.start()].strip()

#                         if (description[-1] == '~'):
#                             prev_desc = description.replace('~', '')
#                             desc_list = prev_desc.split(' ')
#                             number = desc_list[-1]
                            
#                             description = description.replace(f' {number}~', '')
#                             price = number + '~' + price

#                     else:
#                         price = '가격 정보 없음'
#                         description = menu_item.strip()

#                     menus.append({'menu': description, 'price': price})

#             else:
#                 menus.append({'menu': '메뉴 정보 없음', 'price': '가격 정보 없음'})

#             # 레스토랑 정보 추가
#             restaurants.append({
#                 'Name': name,
#                 'Category': category,
#                 'Phone': phone,
#                 'Address': address,
#                 'Image': image,
#                 'Menu': menus
#             })

# # JSON 파일로 저장
# with open('restaurants_info.json', 'w', encoding='utf-8') as f:
#     json.dump(restaurants, f, ensure_ascii=False, indent=4)

# print("크롤링한 데이터가 'restaurants_info.json' 파일에 저장되었습니다.")

In [1]:
import requests
import json
import re
import mariadb
import time

# 데이터베이스 연결 설정
try:
    db_connection = mariadb.connect(
        host="host.docker.internal",
        port=3306,         # MariaDB 기본 포트
        user="boot",
        password="boot",
        database="boot_db"
    )
    print("Database connection successful")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB: {e}")

cursor = db_connection.cursor()

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

# 최대 재시도 횟수
max_retries = 5

# 페이지별로 데이터 요청 및 처리
for page in range(1, 9):
    url = f'https://map.naver.com/p/api/search/allSearch?query=%EB%8F%99%EA%B5%AD%EB%8C%80+%EC%A3%BC%EB%B3%80+%EC%9D%8C%EC%8B%9D%EC%A0%90&type=all&searchCoord=126.97825%3B37.566551&page={page}'
    
    for attempt in range(max_retries):
        res = requests.get(url, headers=headers)
        print(f"페이지 {page} 요청 상태 코드: {res.status_code}")
        
        if res.status_code == 503:
            print(f"503 오류 발생, {attempt + 1}번째 재시도 중...")
            time.sleep(10)
            continue
        
        try:
            data = res.json()
            break
        except json.JSONDecodeError:
            print("응답을 JSON으로 파싱할 수 없습니다.")
            print(f"응답 내용: {res.text}")
            data = None
            continue

    if data is None:
        print(f"페이지 {page} 데이터 요청 실패. 다음 페이지로 넘어갑니다.")
        continue

    # 음식점 정보 추출 및 DB 삽입
    if 'result' in data and 'place' in data['result']:
        for item in data['result']['place']['list']:
            name = item.get('name')
            categories = item.get('category', [])
            category = categories[1] if categories and categories[0] == '음식점' and len(categories) > 1 else categories[0] if categories else '카테고리 없음'
            phone = item.get('tel', '전화번호 없음')
            address = item.get('roadAddress', '주소 없음')
            image_url = item.get('thumUrl', '이미지 없음')

            # 음식점 정보를 DB에 삽입
            cafeteria_insert_query = """
            INSERT INTO cafeteria (cafeteria_name, cafeteria_category, cafeteria_phone, cafeteria_address)
            VALUES (%s, %s, %s, %s)
            """
            cursor.execute(cafeteria_insert_query, (name, category, phone, address))
            cafeteria_id = cursor.lastrowid  # 방금 삽입한 음식점의 ID

            # 이미지 정보를 DB에 삽입
            image_insert_query = """
            INSERT INTO image (creator_id, stored_file_path, is_user_image)
            VALUES (%s, %s, %s)
            """
            cursor.execute(image_insert_query, (cafeteria_id, image_url, False))
            image_id = cursor.lastrowid  # 방금 삽입한 이미지의 ID

            # 음식점의 이미지 ID 업데이트
            update_cafeteria_query = """
            UPDATE cafeteria
            SET cafeteria_image_id = %s
            WHERE id = %s
            """
            cursor.execute(update_cafeteria_query, (image_id, cafeteria_id))

            # 메뉴 정보가 있는 경우
            menu_info = item.get('menuInfo', '')
            if menu_info:
                menu_list = menu_info.split('|')
                for menu_item in menu_list:
                    match = re.search(r'(\d{1,3}(?:,\d{3})*)$', menu_item.strip())
                    if match:
                        price = match.group(0)
                        description = menu_item[:match.start()].strip()
                        if description[-1] == '~':
                            prev_desc = description.replace('~', '')
                            desc_list = prev_desc.split(' ')
                            number = desc_list[-1]
                            description = description.replace(f' {number}~', '')
                            price = number + '~' + price
                    else:
                        price = '가격 정보 없음'
                        description = menu_item.strip()

                    menu_insert_query = """
                    INSERT INTO menu (cafeteria_id, menu_name, menu_price)
                    VALUES (%s, %s, %s)
                    """
                    cursor.execute(menu_insert_query, (cafeteria_id, description, price))
            else:
                # 메뉴 정보가 없을 경우 기본 값을 삽입
                menu_insert_query = """
                INSERT INTO menu (cafeteria_id, menu_name, menu_price)
                VALUES (%s, %s, %s)
                """
                cursor.execute(menu_insert_query, (cafeteria_id, '메뉴 정보 없음', '가격 정보 없음'))

# 트랜잭션 커밋
db_connection.commit()

# 데이터베이스 연결 종료
cursor.close()
db_connection.close()

print("크롤링한 데이터가 데이터베이스에 성공적으로 삽입되었습니다.")

Database connection successful
페이지 1 요청 상태 코드: 200
페이지 2 요청 상태 코드: 503
503 오류 발생, 1번째 재시도 중...
페이지 2 요청 상태 코드: 503
503 오류 발생, 2번째 재시도 중...
페이지 2 요청 상태 코드: 503
503 오류 발생, 3번째 재시도 중...
페이지 2 요청 상태 코드: 503
503 오류 발생, 4번째 재시도 중...
페이지 2 요청 상태 코드: 503
503 오류 발생, 5번째 재시도 중...
페이지 3 요청 상태 코드: 503
503 오류 발생, 1번째 재시도 중...
페이지 3 요청 상태 코드: 503
503 오류 발생, 2번째 재시도 중...
페이지 3 요청 상태 코드: 503
503 오류 발생, 3번째 재시도 중...
페이지 3 요청 상태 코드: 503
503 오류 발생, 4번째 재시도 중...
페이지 3 요청 상태 코드: 503
503 오류 발생, 5번째 재시도 중...
페이지 4 요청 상태 코드: 503
503 오류 발생, 1번째 재시도 중...
페이지 4 요청 상태 코드: 503
503 오류 발생, 2번째 재시도 중...
페이지 4 요청 상태 코드: 503
503 오류 발생, 3번째 재시도 중...
페이지 4 요청 상태 코드: 503
503 오류 발생, 4번째 재시도 중...
페이지 4 요청 상태 코드: 503
503 오류 발생, 5번째 재시도 중...
페이지 5 요청 상태 코드: 503
503 오류 발생, 1번째 재시도 중...
페이지 5 요청 상태 코드: 503
503 오류 발생, 2번째 재시도 중...
페이지 5 요청 상태 코드: 503
503 오류 발생, 3번째 재시도 중...
페이지 5 요청 상태 코드: 503
503 오류 발생, 4번째 재시도 중...
페이지 5 요청 상태 코드: 503
503 오류 발생, 5번째 재시도 중...
페이지 6 요청 상태 코드: 503
503 오류 발생, 1번째 재시도 중...
페이지 6 요청 상태 코드: 503
503 오